# Building an Advanced RAG System with AI21's Jamba-1.5-large

This notebook demonstrates implementing a Retrieval Augmented Generation (RAG) system using AI21's Jamba-1.5-large language model. Jamba-1.5-large features a 256k token context window, making it highly effective for RAG applications by allowing:

- Processing of larger chunks of retrieved content
- Better handling of long-form context
- More comprehensive document analysis

We'll combine this with vector storage and embeddings to create an efficient information retrieval and generation pipeline.

In [1]:
from langchain_ai21 import ChatAI21

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [7]:
import getpass
import os

if not os.environ.get("AI21_API_KEY"):
  os.environ["AI21_API_KEY"] = getpass.getpass("Enter API key for Jamba: ")

from langchain_ai21 import ChatAI21

llm =ChatAI21(model="jamba-1.5-large")

In [1]:
from langchain_voyageai import VoyageAIEmbeddings

In [11]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()


api=os.getenv("VOYAGE_API_KEY")


embeddings = VoyageAIEmbeddings(model="voyage-3-large",api_key=api)




In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)